In [56]:
! ls -alh /var/genie-in-the-box/src/conf/long-term-memory/solutions

total 52K
drwxr-xr-x 2 1000 1000 4.0K Aug 18 15:48 .
drwxr-xr-x 3 1000 1000 4.0K Aug 17 22:53 ..
-rw-r--r-- 1 1000 1000  41K Aug 18 15:39 what-day-is-today-0.json


In [57]:
# ! pip install python-dotenv
! pip install langchain
! pip install langchain[docarray]
! pip install tiktoken
! pip install pinecone-client
! pip install jq
! pip install chromadb
! pip install openai[embeddings]


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [67]:
import sys
import os

import pandas as pd
import json

path = "/var/genie-in-the-box/src/lib"
if path not in sys.path:
    sys.path.append( path )

import util as du
import util_stopwatch as sw
import solution_snapshot_mgr as ssm
import solution_snapshot as ss

path = "/var/genie-in-the-box/src"
du.add_to_path( path )
sys.path

Path [/var/genie-in-the-box/src] already in sys.path


['/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug',
 '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev',
 '/var',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/var/genie-in-the-box/src/lib',
 '/var/genie-in-the-box/src/lib',
 '/var/genie-in-the-box/src']

In [68]:
timer = sw.Stopwatch()
timer.print( "Foo completed!", use_millis=True )

Foo completed! in 0 ms



In [69]:
# reload source code using inline magic for Jupiter notebook
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
%autoreload
path_to_snapshots = os.path.join( du.get_project_root(), "src/conf/long-term-memory/solutions/" )
ss_manager = ssm.SolutionSnapshotManager( path_to_snapshots )
print( ss_manager )

[1] snapshots by question loaded from [/var/genie-in-the-box/src/conf/long-term-memory/solutions/]


In [71]:
from langchain.agents.agent_toolkits.pandas.base import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [63]:
# Load CSV data From file into the Pandas DataFrame.
df = pd.read_csv( du.get_project_root() + "/src/conf/long-term-memory/events.csv" )
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   start_date                  200 non-null    object
 1   end_date                    200 non-null    object
 2   start_time                  200 non-null    object
 3   end_time                    200 non-null    object
 4   event_type                  200 non-null    object
 5   recurrent                   200 non-null    bool  
 6   recurrence_interval         47 non-null     object
 7   priority_level              200 non-null    object
 8   name                        200 non-null    object
 9   relationship                200 non-null    object
 10  description_who_what_where  200 non-null    object
dtypes: bool(1), object(10)
memory usage: 15.9+ KB


In [64]:
def get_event_count( df, event ):

    print( "Counting [{}] events...".format( event ), end="\n\n" )

    count = df[ df.event_type == event ].shape[ 0 ]

    print()
    print( "Done! Total {} {} events".format( count, event ) )

get_event_count( df, "Birthday" )

Counting [Birthday] events...


Done! Total 16 Birthday events


In [105]:
event_prompt_template_string = """
You are an event planner with proven capacity for querying pandas dataframes.
The dataframe that I am providing you for inspection contains two important fields:
1) event_type: tells us what kind of event is taking place, such as birthday, conference, meetings, etc.
2) description_who_what_where: tells us who, what and/or where the event is about, as in 'my friend's birthday at their favorite bar', the name of a conference, or what a meeting is about, e.g.: 'yet another boring meeting with the boss'.
I expect you to query the dataframe provided and answer in plain English unless specifically requested to do otherwise.
I expect you to remember query techniques and solutions that work, so that you do not have to begin from zero.
When you recognize a new query as similar to a previous query, say so
NEVER display raw dataframe content, unless specifically requested to do otherwise.
All dates and temporal references like 'today', 'yesterday', 'next week' should all be calculated by using Python's datetime library.
When I ask you about events contained in this dataframe, you will always sort the events in ascending order, 1 per line, unless specifically requested to do otherwise.
When summarizing your results you will include the event type, start time, date (e.g.: today, tomorrow, Wednesday the 10th, etc.), and what the event is about, using conversational English, unless specifically requested to do otherwise.
DO NOT prepend numbers to a list of events, unless specifically requested to do otherwise.
If the question, delimited by three hash symbols ###, is vague or difficult to answer, please ask for clarification.

{history}

Given these instructions, answer the following question: ###{question}###

Format your response as a json object containing the following fields
1) question: the question that was asked
2) answer: the answer to the question
3) solution_summary: a plain english summary of the solution used to answer the question
4) code: the code used to answer the question, formatted as a list of strings, one string per line.
5) programming_language: the programming language used to answer the question
6) language_version: the programming language version used to answer the question
"""

solution_history_string = """
You have successfully answered either this exact question or a question similar to this before.

This is your summary of the solution:

{solution_summary}

This is the code you used to answer the question:

{solution_code}

Please examine the summary and the code and then modify the code if you think it is necessary.
Once you have done that, execute the code. If it fails, debug it.
"""

event_prompt_template     = ChatPromptTemplate.from_template( event_prompt_template_string )
# solution_history_template = ChatPromptTemplate.from_template( solution_history_string )

In [106]:
def get_history( question ):

    similar_snapshots = ss_manager.get_snapshots_by_question( question )

    if len( similar_snapshots ) > 0:

        solution_summary = similar_snapshots[ 0 ][ 1 ].solution_summary
        solution_code    = "\n".join( similar_snapshots[ 0 ][ 1 ].code )

        solution_history = solution_history_string.format( solution_summary=solution_summary, solution_code=solution_code )
    else:
        solution_history = ""

    return solution_history

question = "What day is today?"
# question = "Why is the sky blue?"

# solution_summary, solution_code = get_history( question )
solution_history = get_history( question )

solution_history

get_snapshots_by_question( 'what day is today' )...
Exact match: Snapshot with question [what day is today] exists!
Found [1] similar snapshots
score [100.0] for [what day is today] == [what day is today]


"\nYou have successfully answered either this exact question or a question similar to this before.\n\nThis is your summary of the solution:\n\nI used Python's datetime library to get the current date.\n\nThis is the code you used to answer the question:\n\nimport datetime\ntoday = datetime.date.today()\ntoday\n\nPlease examine the summary and the code and then modify the code if you think it is necessary.\nOnce you have done that, execute the code. If it fails, debug it.\n"

In [107]:
def get_event_prompt( question ):

    question = ss.SolutionSnapshot.clean_question( question )
    history  = get_history( question )

    event_prompt = event_prompt_template.format_messages( question=question, history=history )

    return event_prompt

event_prompt = get_event_prompt( "What day is today?" )
# event_prompt = get_event_prompt( "Why is the sky blue?" )

print( event_prompt, end="\n\n" )

get_snapshots_by_question( 'what day is today' )...
Exact match: Snapshot with question [what day is today] exists!
Found [1] similar snapshots
score [100.0] for [what day is today] == [what day is today]
[HumanMessage(content="\nYou are an event planner with proven capacity for querying pandas dataframes.\nThe dataframe that I am providing you for inspection contains two important fields:\n1) event_type: tells us what kind of event is taking place, such as birthday, conference, meetings, etc.\n2) description_who_what_where: tells us who, what and/or where the event is about, as in 'my friend's birthday at their favorite bar', the name of a conference, or what a meeting is about, e.g.: 'yet another boring meeting with the boss'.\nI expect you to query the dataframe provided and answer in plain English unless specifically requested to do otherwise.\nI expect you to remember query techniques and solutions that work, so that you do not have to begin from zero.\nWhen you recognize a new qu

In [28]:
llm      = ChatOpenAI( model_name="gpt-4-0613", temperature=0.0 )
df_agent = create_pandas_dataframe_agent( llm, df, verbose=True )

In [108]:
# Create function to take a question and call the data frame agent.
def query_dataframe( question ):

    timer = sw.Stopwatch()

    # event_prompt = event_prompt_template.format_messages( question=question )
    event_prompt = get_event_prompt( question )
    response = df_agent.run( event_prompt )

    timer.print( "Query completed!", use_millis=True )

    return response

response = query_dataframe( "What day is today?" )
# response = query_dataframe( "What day was yesterday?" )
# response = query_dataframe( "What day is tomorrow?" )

response

get_snapshots_by_question( 'what day is today' )...
Exact match: Snapshot with question [what day is today] exists!
Found [1] similar snapshots
score [100.0] for [what day is today] == [what day is today]


> Entering new AgentExecutor chain...
Thought: The question is asking for the current date. I can use the datetime library in Python to get this information. The solution summary and code provided are correct for this question, so no modification is necessary.
Action: python_repl_ast
Action Input: import datetime
today = datetime.date.today()
today
Observation: 2023-08-18
Thought:The current date is 2023-08-18. Now I will format the response as a json object.
Final Answer: {"question": "what day is today", "answer": "2023-08-18", "solution_summary": "I used Python's datetime library to get the current date.", "code": ["import datetime", "today = datetime.date.today()", "today"], "programming_language": "Python", "language_version": "3.7"}

> Finished chain.
Query completed! in 22,42

'{"question": "what day is today", "answer": "2023-08-18", "solution_summary": "I used Python\'s datetime library to get the current date.", "code": ["import datetime", "today = datetime.date.today()", "today"], "programming_language": "Python", "language_version": "3.7"}'

In [109]:
response

'{"question": "what day is today", "answer": "2023-08-18", "solution_summary": "I used Python\'s datetime library to get the current date.", "code": ["import datetime", "today = datetime.date.today()", "today"], "programming_language": "Python", "language_version": "3.7"}'

In [110]:
# parse the json string into a Python dictionary
response_dict = json.loads( response )
response_dict

{'question': 'what day is today',
 'answer': '2023-08-18',
 'solution_summary': "I used Python's datetime library to get the current date.",
 'code': ['import datetime', 'today = datetime.date.today()', 'today'],
 'programming_language': 'Python',
 'language_version': '3.7'}

In [98]:
raw_summary = response[ "solution_code" ]
summary = raw_summary.split( "\n" )
summary

TypeError: string indices must be integers

In [99]:
foo = [ "import datetime", "today = datetime.date.today()", "today" ]

# Get foo as a json string
import json
foo_json = json.dumps( foo )
foo_json

'["import datetime", "today = datetime.date.today()", "today"]'

In [40]:
foo

['import datetime', 'today = datetime.date.today()', 'today']